In [ ]:
from IPython.display import clear_output
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
import pandas as pd
import numpy as np

# Общая идея

Модели машинного обучения представляют собой черный ящик.

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/gan/blackbox.png" alt="alttext" width=400/>


Иногда это становится препятствием для внедрения моделей.

Есть как минимум три причины по которым нас может интересовать объяснение предсказаний модели.

### Поиск ошибок.

Модель может использовать совсем не те признаки, которые соответствуют реальным объектам.

Например ориентироваться на фон или водяной знак, а не на реальные свойства объекта.

Пример из статьи ["Why Should I Trust You?"](https://arxiv.org/abs/1602.04938) 
Авторы обучили классификатор волков и эскимосских собак (хаски). Исследователи на изображениях, отобранных так, чтобы на всех фотографиях волков на фоне был снег, а на фотографиях хаски — нет.


<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/2_bad_models_prediction.png" alt="alttext" width=400/>







### Доверие пользователей.



<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/gan/nuclear.jpg" alt="alttext" width=400/>

Нельзя остановить ядерную электростанцию или назначить пациенту опасное лечение на основании предсказания «черного ящика». 
Даже маловероятная ошибка в таких случаях будет иметь тяжелые последствия. Поэтому, человек принимающий решение должен понимать на основе каких признаков или симптомов сделанно предсказание.



### Публикации в научных журналах

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/gan/scopus.png" alt="alttext" width=400/>

Вероятность публикации статьи значительно повышается, если автор смог объяснить происхождение результатов своего исследования.

## Explainability & Interpretability
В англоязычной литературе можно встретить два термина связанных с темой доверия: Explainability и Interpretability

**Explainability** &mdash; методики позволяющие объяснить механизм функционирования модели. 

Например, для линейной регрссии это анализ коэффицентов при параметрах.


**Interpretability** &mdash; Анализ того как изменение входов модели влияеи на ее выходы.

Например, закрашивая часть пикселей изображения, можно выяснит какие из них повлияют на предсказания (пример с хаски и волками).


В данном блокноте будем приемущественно рассматривать методы второго типа.

[Machine Learning Explainability vs Interpretability: Two concepts that could help restore trust in AI](https://www.kdnuggets.com/2018/12/machine-learning-explainability-interpretability-ai.html)



##### $\color{brown}{\text{Допольнительная информация}}$




Машинное обучение лежит в основе многих последних достижений в области науки и технологии.

Когда компьютеры победили профессионалов в таких играх, как Go, многие люди начали спрашивать: могут ли машины стать лучшими водителями или даже лучшими врачами?

Во многих приложениях машинного обучения пользователей просят доверять модели, которая поможет им принимать решения. Врач определенно не станет оперировать пациента просто потому, что «так сказала модель».

Даже в ситуациях с более низкими рисками, принять неправильное решение на основании предсказания модели — например, при выборе фильма для просмотра в Netflix, требуется определенная степень доверия.

Несмотря на то, что многие модели машинного обучения представляют собой черный ящик, понимание причин, лежащих в основе прогнозов модели, безусловно, поможет пользователям решить, когда им доверять, а когда нет.

К сожалению, компьютеры обычно не объясняют своих прогнозов. 
И это становится препятствием для внедрения моделей машинного обучения.
А если пользователи не доверяют модели или прогнозу, то они будут с опаской использовать их на практике.

Поэтому вопрос в том, как помочь пользователям доверять результатам, полученным с помощью моделей машинного или глубокого обучения.

Важно различать два разных (но связанных) определения доверия: 
1. Доверие к предсказанию, то есть доверяет ли пользователь отдельному предсказанию настолько, чтобы предпринять какие-то действия на его основе?
2. Доверие к модели, то есть доверяет ли пользователь модели так, чтобы применять ее на практике?

Оба напрямую подвержены влиянию того, насколько человек понимает поведение модели, чтобы не рассматривать ее как черный ящик. 

Определение доверия к индивидуальным прогнозам является важной задачей, когда модель используется для принятия решений. Ведь, например, при медицинской диагностике или обнаружении террористов предсказания не могут быть выполнены бездумно, так как последствия могут быть катастрофическими.

Помимо доверия индивидуальным предсказаниям, необходимо доверять модели "в целом". Чтобы принять это решение, пользователи должны быть уверены, что модель будет хорошо работать на реальных данных согласно интересующим метрикам. Реальные данные часто значительно отличаются, и, кроме того, метрика оценки может не указывать на цель продукта. Изучение индивидуальных прогнозов и их объяснение — важная задача, которая может помочь пользователям, особенно для больших наборов данных.

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/impact_assessment.png" alt="alttext" width=450/> 

Практики часто **переоценивают точность своих моделей**. Примером тому может послужить известный случай с моделью, обученной отличать хаски от волка.

Часто **артефакты сбора данных могут вызвать нежелательные корреляции**, которые классификаторы улавливают во время обучения. Эти проблемы может быть очень трудно определить, просто взглянув на необработанные данные и прогнозы.

Пример идеально решенной задачи, не применимой на практике — задача различить на фотографиях волков и эскимосских собак (хаски). Исследователи обучили классификатор логистической регрессии из 20 изображений, отобранных вручную так, чтобы на всех фотографиях волков на фоне был снег, а на фотографиях хаски — нет.

Этот эксперимент демонстрирует полезность объяснения отдельных визуальных признаков для понимания работы классификаторов перед тем, как принять решения о доверии к ним.


<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/2_bad_models_prediction.png" alt="alttext" width=400/>

["Why Should I Trust You?": Explaining the Predictions of Any Classifier](https://arxiv.org/abs/1602.04938)


###### Критерии, которым должны отвечать Интерпретаторы

1. **Результат должен быть интерпретируемым**, т. е. обеспечивать качественное понимание взаимосвязи между входными переменными и ответом. Отметим, что интерпретируемость должна учитывать ограничения пользователя. Объяснения должны быть легкими для понимания, не обязательно соответствовать функциям, используемым моделью. Ну и конечно, понятие интерпретируемости зависит от целевой аудитории.

Например, возможное интерпретируемое представление для классификации текста — это двоичный вектор, обозначающий наличие или отсутствие слова, даже если классификатор может использовать более сложные (и непонятные) функции, такие как вложения слов. Аналогичным образом для классификации изображений, интерпретируемое представление может быть двоичным вектором, указывающим «наличие» или «отсутствие» смежного участка аналогичных пикселей (суперпиксель), тогда как классификатор может представлять изображение в виде тензора с тремя цветовыми каналами на пиксель.

2. Еще один важный критерий — **локальная точность**. Чтобы иметь смысл, обьяснение должно соответствовать тому, как модель ведет себя конкретно для предсказываемого случая.

Отметим, что локальная точность не подразумевает глобальной точности: факторы, которые важны в глобальном масштабе, могут не иметь значения на местном контексте, и наоборот. 
В то время как глобальная точность не всегда будет означать локальную, выделяя глобально достоверные объяснения сложных моделей, которые сложно интерпретировать для конкретного примера.


Некоторые метрики, такие как accuracy, **часто могут быть неподходящей метрикой для оценки модели**. Например, когда речь идет о несбалансированных датасетах.

Другой пример: модель прогнозирования оттока клиентов: модель может сказать вам, что конкретный клиент с вероятностью 90% откажется от услуг, но без четкого понимания причины не ясно, что можно сделать, чтобы предотвратить отток.

Самая точная модель в мире бесполезна, если она не используется для принятия решений и действий.

Поэтому крайне важно сделать модель максимально прозрачной и понятной для заинтересованных сторон, чтобы ее можно было использовать и действовать соответствующим образом.


Таким образом в процесс обучения обучения добавляется пункт **Оценка результата**


<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/gan/3_1_structure.png" alt="alttext" width=750/>

Для этой задачи существуют специальные библиотеки (LIME, SHAP).





# Оценка важности признаков в простых случаях

In [ ]:
from IPython.display import clear_output
import pandas as pd
import numpy as np

В случае с линейными моделями нам было сравнительно легко определить, какие признаки модель считает важными $-$ если модель присваивает какому-то признаку большой вес, то скорее всего это признак достаточно "хороший".


Для примера скачаем датасет жилья Бостона (boston_dataset), в котором проанализируем зависимость цены на жилье от параметров жилья и района, в котором оно находится.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# load dataset
boston_dataset = load_boston()
X = pd.DataFrame(data=boston_dataset['data'], columns=boston_dataset['feature_names'])
y = boston_dataset['target']

# Можно посмотреть детальное описание датасета
print(boston_dataset.DESCR)

Обучим модель

In [ ]:
model = LinearRegression()
model.fit(X, y)

Выведем коэффициенты признаков

In [ ]:
model.coef_

In [ ]:
df = pd.DataFrame({"name": X.columns, "coef": model.coef_})

plt.figure(figsize=(8,8))
sns.barplot(data=df, y="name", x="coef", color="blue", orient = 'h')
plt.show()

## Оценка важности признака для дерева

В случае же с деревьями всё далеко не так очевидно: дерево не знает такой концепции как "вес признака".

Универсального критерия значимости для деревьев у нас нет, и в зависимости от задачи и от того, как эти признаки устроены, ответы могут быть разными.

Первый из способов посчитать, насколько тот или иной признак значим для дерева это Gini Impurity measure $-$ показывает, насколько хорошо переменная помогает нам разбивать данные. По сути перекликается с Impurity decrease, который мы использовали при построении  самого дерева.

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/pdd/Giny_impurity1.png" alt="alttext" width=600/>

Логика такая: если часто признак у нас присутствует на верхних уровнях дерева, то есть он помогает на маленьких глубинах разбивать выборку, то он более значим, чем признаки, которые используются на более поздних этапах.

Но при этом не учитывается, насколько помогло такое начальное разбиение (вспомним пример с XOR). Хорошо было бы учитывать еще и насколько улучшается качество при таком разбиении. Тут нам поможет дополнительно учитывать как раз impurity decrease.

Такая метрика встроена в любое дерево решений. Для случайного леса (и других ансамблей) просто выдается среднее по деревьям.

Однако, у нее есть один существенный **недостаток**: чем больше переменная может принимать значений, тем больше у нас возможных порогов разбиений, следовательно, тем больше вероятность получить "хорошее" разбиение. 
Таким образом из двух одинаково неинформативных признаков преимущество будет получать тот, у которого возможных разбиений больше.

Бинарные признаки и маломерные вещественные в любом случае будут получать заниженное качество по сравнению с вещественными, даже если те дают худшее разбиение.

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/pdd/Giny_impurity2.png" alt="alttext" width=600/>

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rng = np.random.RandomState(42)
model = RandomForestRegressor(random_state=rng)
model.fit(X, y)

In [ ]:
model.feature_importances_

In [ ]:
df = pd.DataFrame({"name": X.columns, "imp": model.feature_importances_})

plt.figure(figsize=(8,8))
sns.barplot(data=df, y="name", x="imp", color="blue", orient = 'h')
plt.show()


## Randomization/Permutation



Мы можем перемешать значения нашей переменной. За счет этого она станет бессмысенной. По падению качества мы можем оценить, каков был вклад переменной до изменения: если качество упало сильно, то признак был значимым.

Можно считать качество на OOB $-$ **Out-of-bag samples**: объектах, которые были выброшены при бутстрепе и которые модель таким образом еще не видела: таким образом, нам не надо делать отдельную валидацию. По очереди пермешиваем значения каждой переменной в OOB и смотрим, как падает качество.
В sklearn это реализовано как <a href="https://scikit-learn.org/stable/modules/permutation_importance.html">отдельный класс</a>. 

Однако, такой подход в чистом виде приведет к завышению важности для коррелирующих признаков.

https://blog.ceshine.net/post/please-stop-permuting-features/




Для дерева решений это объясняется следующим образом: линейная регрессия от двух сильно скоррелированных признаков (один линейно выражается через другой) по сути представляет собой регрессию от одного признака. 

Дерево решений пытается разбить пространство плоскостями, и в областях, где объектов нет, оно по сути занимается угадыванием. Если x1 и x2 на картинке ниже линейно зависимы, то не может возникнуть ситуация, при которой x1 = 0, а x2 = 1. А как раз при перемешивании такая ситуация возникнет, и точки начнут попадать в "проблемные" области, в которых дерево решений плохо предсказывает.

В результате получаем завышенную важность.

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/pdd/figure-4.png" alt="alttext" width=800/>

https://arxiv.org/abs/1905.03151

Второй вариант $-$ тоже считать качество на OOB, но при этом при каждом перемешивании тренировать модель **заново**. Это работает адекватно, но очень долго.

А еще можно пытаться перемешивать так, чтобы не разрушать связи между скоррелированными признаками. В этом случае мы не будем часто попадать в регионы, на которых у нас заведомо не может быть объектов. Также можно добавить сюда переучивание модели каждый раз.  

In [ ]:
from sklearn.inspection import permutation_importance

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=rng)
model = RandomForestRegressor(random_state=rng)
model.fit(X_train, y_train)

r = permutation_importance(model, X_val, y_val, n_repeats=100, random_state=rng)
r.importances_mean

In [ ]:
df = pd.DataFrame({"name": X.columns, "imp": r.importances_mean})

plt.figure(figsize=(8, 8))
sns.barplot(data=df, y="name", x="imp", color="blue", orient = 'h')
plt.show()

## Dropped variable importance

Редко используемый метод, заключающийся в том, что мы просто выбрасываем переменную и смотрим как упало качество по сравнению с "полной" моделью.

Это не очень хорошо как минимум потому, что многие гиперпараметры зависят от числа признаков $-$ модель будет работать хуже еще и поэтому. Опять же между признаками могут быть сложные взаимодействия.

# Библиотеки для реализации explanation

Мы рассмотрим две библиотеки: SHAP и LIME.

Логика работы **SHAP (SHapley Additive exPlanations)** основана на значении Шепли.
[Значение Шепли](https://en.wikipedia.org/wiki/Shapley_value) — это среднее значение предельных вкладов по всем перестановкам.

SHAP учитыват все возможные перестановки, таким образом, SHAP представляет собой единый подход, обеспечивающий глобальную и локальную согласованность и интерпретируемость.

Однако его цена — время, так как алгоритму нужно вычислить все перестановки, чтобы получить результаты.

Напротив, **LIME (Local Interpretable Model-agnostic Explanations)** строит дискретные линейные модели вокруг индивидуального прогноза в его локальной окрестности. LIME на самом деле является подмножеством SHAP, но не имеет тех же свойств.

**Преимущество LIME — скорость**.
Алгоритм LIME изменяет данные вокруг отдельного прогноза для построения модели, в то время как SHAP должен вычислять все перестановки глобально, чтобы получить локальную точность. Кроме того, модуль SHAP Python пока еще не имеет специально оптимизированных алгоритмов для всех типов алгоритмов (таких как KNN).

## [SHAP (SHapley Additive exPlanations)](https://christophm.github.io/interpretable-ml-book/shap.html)



<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/4_shap_diagram.png" alt="alttext" width=400/>


Цель SHAP — объяснить предсказание экземпляра x путем вычисления вклада каждой функции в предсказание. Для этого вычисляются значения Шепли из теории игр. А значения признаков экземпляра данных действуют как игроки в коалиции.

Значения Шепли говорят нам, как справедливо распределить «выплату» (= прогноз) между функциями. 
* Игрок может иметь индивидуальное значение функции, например для табличных данных. 
* Или может быть группой значений характеристик. Например, для объяснения изображения пиксели могут быть сгруппированы в суперпиксели, а прогноз распределяется между ними.




$g(z^{'})=\phi_{0} + \sum_{j=1}^{M}\phi_{j}z^{'}_{j}$

где g — модель объяснения
<math xmlns="http://www.w3.org/1998/Math/MathML">
  <msup>
    <mi>z</mi>
    <mo>&#x2032;</mo>
  </msup>
  <mo>&#x2208;<!-- ∈ --></mo>
  <mo fence="false" stretchy="false">{</mo>
  <mn>0</mn>
  <mo>,</mo>
  <mn>1</mn>
  <msup>
    <mo fence="false" stretchy="false">}</mo>
    <mi>M</mi>
  </msup>
</math> — вектор коалиции, M — максимальный размер коалиции и
<math xmlns="http://www.w3.org/1998/Math/MathML">
  <msub>
    <mi>&#x03D5;<!-- ϕ --></mi>
    <mi>j</mi>
  </msub>
  <mo>&#x2208;<!-- ∈ --></mo>
  <mrow class="MJX-TeXAtom-ORD">
    <mi mathvariant="double-struck">R</mi>
  </mrow>
</math> — атрибуция признака j, значения Шепли.

Таким образом, например, изображения не представлены на уровне пикселей, а агрегируются в суперпиксели.

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/pdd/shap1.png" alt="alttext" width=500/>


https://towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30

Пусть у нас есть некая модель, которая предсказывает з/п на основании пола, возраста и работы. 

Самое простое "предсказание" это средняя зарплата $-$ не учитывает никакие признаки (точность соотвествующая).

Дальше мы можем использовать по однуму признаку, более сложные модели базируются на многих признаках.

Зная предсказания всех возможных моделей, мы можем посчитать *вклад* каждого признака в общее предсказание.

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/pdd/shap2.png" alt="alttext" width=480/>

На графе вложенности моделей каждое ребро сооветствует добавлению какого-то признака. Например, на картинке выше все покрашенные ребра добавляют признак "возраст" к моделям.

Дальше мы можем взвесить все ребра так, чтобы на каждом уровне сумма весов по "строке" была постоянной (см ниже). Инициализируем веса одинаковыми числами (например, долями единицы).

Дальше проходим по графу и при переходе по ребрам домножаем вес ребра на разницу предсказания по данному ребру и суммируем все такие ребра для данного признака (в данном случае возраст):

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/pdd/shap3.png" alt="alttext" width=600/>

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/pdd/shap4.png" alt="alttext" width=600/>


Такие Shap Values можно легко посчитать для каждого признака для каждого объекта и, например, усреднить по объектам какой-то группы. 

Например, можно взять все объекты, которые модель скорит высоко, и посмотреть значимые дли них признаки. Либо, если у нас есть какая-то особенная группа объектов, можем контролировать, что для них модель выучивает именно важные признаки, а не какой-то мусор.

Посмотрим, как реализуется этот подход в NLP

In [ ]:
from IPython.display import clear_output

Установим пакета SHAP 

In [ ]:
!pip install shap
clear_output()

import shap

#### Пример обьяснения перевода с английского на русский
Рассмотрим пример интерпретации модели для предварительно обученной модели машинного перевода
[Machine Translation Example](https://shap.readthedocs.io/en/stable/example_notebooks/text_examples/translation/Machine%20Translation%20Explanation%20Demo.html). И, раз уж мы будем для перевода использовать предобученную модель-транформер, то переведем начало статьи [How Transformers Work](https://towardsdatascience.com/transformers-141e32e69591)

##### $\color{brown}{\text{Допольнительная информация}}$

О трансформерах

Attention

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-020.png" width="700">


Self - attention
<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-070.png" width="700">

- Реккурентность не нужна, можем обрабатывать данные параллельно.

Transformer

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-092.png" width="700">



### NLP модель

Используем одну из моделей [huggingface ](https://github.com/huggingface/transformers)


Модель: https://blogs.helsinki.fi/language-technology/ 


https://huggingface.co/Helsinki-NLP https://huggingface.co/Helsinki-NLP/opus-mt-en-ru

In [ ]:
!pip install sentencepiece
!pip install transformers
clear_output()

Загружаем модель.

Для этого используется класс-фабрика, на вход которому передается имя модели, а возвращается он объект соответствующего класса.

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from transformers import  AutoModelForSeq2SeqLM
import sentencepiece
lang = "en"
target_lang = "ru"
model_name = f'Helsinki-NLP/opus-mt-{lang}-{target_lang}'

# Download the model and the tokenizer
# Можно попробовать перевод и разными предобученными моделями


# It's a Factory pattern
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)
clear_output()
print(type(model))

В данном случае нам вернулся объект типа [MarianMT](https://huggingface.co/transformers/model_doc/marian.html)



Теперь создадим [токенайзер](https://huggingface.co/transformers/main_classes/tokenizer.html).

Как мы уже обсуждали, токенайзер преобразует слова и знаки препинания в исходного текста в токены которые можно подать на вход модели. В данном случае возвращаются id. При этом не всегда одно слово преобразуется в один токен, иногода слово разбивается по слогам на несколько токенов.


Создается токенайзер так же фабрикой, по имени модели.


In [ ]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_name)
print(type(tokenizer))

inputs = tokenizer("Hello world!", return_tensors="pt")
print(inputs)

translated = model.generate(**tokenizer("Hello world!", return_tensors="pt").to(device))
 # ** -  is dictionary unpack operator
 # https://towardsdatascience.com/unpacking-operators-in-python-306ae44cd480

Теперь переведем целую фразу. 

И проанализируем как выход модели связан со входом.

Для этого создадим объект [shap.Explaner](https://shap.readthedocs.io/en/latest/generated/shap.Explainer.html)

Который в данном случае инициализируется экземпляром модели и экземпляром токенайзера*.


Вместо того что бы запускать саму модель, мы запускаем Explainer (неявно вызывая его метод __call__). 

* В действительности вторым параметром конструктора shap.Explainer не обязательно должен быть токенайзер. shap.Explainer принимает объект поддерживающий интерфейс masker:

masked_args = masker(*model_args, mask=mask)

Он используется для исключения части аргументов и токенайзеры поддерживают этот интерфейс (shap.TokenMasker). Благодаря такому подходу shap может работать с k.,с различными моделями как с "черным ящиком", 





In [ ]:
import shap
# define the input sentences we want to translate
data = [
    "Transformers are a type of neural network architecture that have been gaining popularity. Transformers were developed to solve the problem of sequence transduction, or neural machine translation. That means any task that transforms an input sequence to an output sequence. This includes speech recognition, text-to-speech transformation, etc.."
]

# we build an explainer by passing the model we want to explain and
# the tokenizer we want to use to break up the input strings
explainer = shap.Explainer(model, tokenizer)

# explainers are callable, just like models
explanation = explainer(data, fixed_context=1)

Вместо того что бы запускать саму модель, мы запускаем экземпляр Explainer (неявно вызывая его метод call). 

На выходе получаем объект класса [shap.Explanation](https://shap.readthedocs.io/en/latest/generated/shap.Explanation.html#shap-explanation)


Который содержит значения Шепли для каждого токена.

In [ ]:
print("Data", explanation.data)
print("Shap values", explanation.values)
print("Shape", explanation.shape) # 1, in, out



Теперь используя магию shap можно визуализировать результат.

In [ ]:
shap.plots.text(explanation)

## LIME

Local Interpretable Model-agnostic Explanations

Поскольку мы хотим быть независимыми от модели, то следующее, что мы можем сделать, чтобы изучить поведение базовой модели — это изменить входные данные и посмотреть, как меняются прогнозы.

Это оказывается преимуществом с точки зрения интерпретируемости, потому что мы можем изменять компоненты на входе, которые имеют смысл для людей (например, слова или части изображения), даже если модель использует гораздо более сложные компоненты в качестве функций (например, word embeddings).

LIME генерирует объяснение, аппроксимируя базовую модель интерпретируемой (например, линейной моделью с несколькими ненулевыми коэффициентами), изученной при возмущениях исходной модели (например, удалении слов или скрытии частей изображения).

Ключевая идея, лежащая в основе LIME, заключается в том, что гораздо проще аппроксимировать модель черного ящика простой моделью локально, чем пытаться обьяснить модель целиком (глобально).



### Принцип работы:


1.   Делаем предсказание для целевого объекта
2.   Убираем часть признаков* у объекта и делаем новое предсказание
3.   Шаг 2 повторяем несколько раз, все предсказания сохраняем
4.   Из полученных данных формируем датасет, на котором обучаем линейную модель.
5.   Коэффициенты линейной модели используем для оценки важности признаков.

Первоисточник: [Why should I trust you?](https://arxiv.org/abs/1602.04938)


### *Как маскировать признаки?

Зависит от типа данных:
*  Для табличных данных достаточно заменить значения признака на None.
*  В текстах можно просто удалить слово. 
*  Для изображений делят картинку на области (суперпиксели) и поочередно закрашивают их одним и тем же цветом (средним)

### Ограничения

Описанный подход не позволит интерпретировать поведение модели только в некоторой области вблизи интересующего нас экземпляра.
На практике этого может быть достаточно. И в отличие от SHAP этот подход работает быстрее так как не требует перебора всех комбинаций признаков.

<img src ="http://edunet.kea.su/repo/src/L10_Explainability/img/lime.png" width="700">



* Модель f &mdash; нелинейная функция, представленная в виде разделяющей поверхности между розовым и голубым фоном.
* Объект, для которого планируеьтся сделать интерпретацию предсказаний модели, обозначен жирным красным крестом
* Сгенерированные объекты обозначены кругами и крестами в зависимости от класса
* Размер объектов отражает их близость к исходному (по некоторой метрике расстояния)
* Пунктир &mdash; граница которую выучила линейная модель

### Пример обьяснения классфикации статей по религиозному принципу


<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/5_christian_or_atheist_2.png" alt="alttext" width=900/>

Используем датасет [fetch_20newsgroups](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html): 

Данные [«The 20 Newsgroups»](http://qwone.com/~jason/20Newsgroups/) — это коллекция примерно из 20000 новостных документов, разделенная (приблизительно) равномерно между 20 различными категориями. Изначально она собиралась Кеном Ленгом (Ken Lang), возможно, для его работы «Newsweeder: Learning to filter netnews» («Новостной обозреватель: учимся фильтровать новости из сети»).

Коллекция «The 20 newsgroups» стала популярным набором данных для экспериментов с техниками машинного обучения для текстовых приложений, таких как классификация текста или его кластеризация.

https://marcotcr.github.io/lime/tutorials/Lime%20-%20multiclass.html

В данном примере мы будем использовать [Multinomial Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html?highlight=multinomial%20naive%20bayes#sklearn.naive_bayes.MultinomialNB) для классификации.

In [ ]:
import sklearn
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
# making class names shorter
class_names = [x.split('.')[-1] if 'misc' not in x else '.'.join(x.split('.')[-2:]) for x in newsgroups_train.target_names]
class_names[3] = 'pc.hardware'
class_names[4] = 'mac.hardware'

print(class_names)

LIME используется для объяснения множества классификаторов (таких как RandomForest или SVM и нейронные сети) при анализе моделей NLP и CV.

Знаменитый набор данных из 20 групп новостей является эталоном и использовался для сравнения различных моделей в нескольких статьях.

Мы берем два класса, которые трудно различить, потому что в них много схожих слов: христианство и атеизм.

Обучая модель, мы получаем точность тестового набора 83,5%, что является удивительно высоким показателем. Если бы точность была нашим единственным мерилом доверия, мы бы точно доверились этому классификатору.

Однако давайте посмотрим на объяснение на рисунке для произвольного экземпляра в тестовом наборе:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.metrics
from sklearn.naive_bayes import MultinomialNB

# Again, let's use the tfidf vectorizer, commonly used for text.
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

# Train the model
nb = MultinomialNB(alpha=.01)
nb.fit(train_vectors, newsgroups_train.target)

# Calculate F1_score
pred = nb.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred, average='weighted')

In [ ]:
print(train_vectors.shape)
print(type(train_vectors[0]), train_vectors[0].shape)
print(vectorizer.get_feature_names())

Как видно из кода, текст подается на вход модели не в сыром виде, а после предобработки объектом vectorizer. 

[LimeTextExplainer](https://lime-ml.readthedocs.io/en/latest/lime.html#lime.lime_text.LimeTextExplainer)  ждет на вход данные и класс модели:


```
explain_instance(text_instance, classifier_fn, labels=(1, ), top_labels=None, num_features=10, num_samples=5000, distance_metric='cosine', model_regressor=None)
```

**classifier_fn** *– classifier prediction probability function, which takes a list of d strings and outputs a (d, k) numpy array with prediction probabilities, where k is the number of classes. For ScikitClassifiers , this is classifier.predict_proba.*



 Что поэтому в примере используется обертка над классом преобразовывающем данные и моделью:

 [sklearn.pipeline.make_pipeline](#https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html)

In [ ]:
from sklearn.pipeline import make_pipeline

#https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html
model_with_preprocessing = make_pipeline(vectorizer, nb)

Мы видим, что этот классификатор имеет очень высокий F1_score. Руководство sklearn для 20 newsgroups указывает, что Multinomial Naive Bayes переучивается на этом наборе данных, изучая нерелевантные взаимосвязи, такие как заголовки.

Теперь мы используем LIME для объяснения индивидуальных прогнозов...


В случае мультикласса мы должны определить, для каких меток хотим получим объяснения с помощью параметра «labels». Сгенерируем пояснения для меток 0 и 17:


In [ ]:
!pip install lime

In [ ]:
import lime
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names, random_state=rng)
idx = 1340
exp = explainer.explain_instance(newsgroups_test.data[idx],
                                 model_with_preprocessing.predict_proba,
                                 num_features=6,
                                 labels=[0,
                                         17
                                         ])
print('Document id: %d' % idx)
print('Predicted class =', class_names[nb.predict(test_vectors[idx]).reshape(1,-1)[0,0]])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

Возвращается как и в случае с SHAP специальный объект класса [Explanation](https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=Explanation#lime.explanation.Explanation)


Попросим LIME сгенерировать метки для K=2 классов. Чтобы увидеть, какие ярлыки имеют объяснения, используйте функцию available_labels

In [ ]:
idx = 1340
exp = explainer.explain_instance(newsgroups_test.data[idx],
                                 model_with_preprocessing.predict_proba,
                                 num_features=6,
                                 top_labels=2)
print(exp.available_labels())

In [ ]:
print(exp.as_list(label = 0))
print(exp.as_list(label = 15))

Обратите внимание, что положительный и отрицательный знаки относятся к конкретной метке — так что слова, отрицательные по отношению к классу 0, могут быть положительными по отношению к классу 15, и наоборот.

In [ ]:
exp.show_in_notebook(text=newsgroups_test.data[idx], labels=(0,))

Теперь давайте посмотрим на визуализацию объяснений.
Обратите внимание, что для каждого класса слова в правой части строки являются «положительными», а слова в левой части — «отрицательными» для обьясняемого класса.

Также видно, что в классификаторе используются как разумные слова (такие как «геноцид», «Лютер», «семитский» и т. д.), так и неразумные («рис», «сова»).

Давайте увеличим масштаб и просто посмотрим на объяснения класса «атеизм».

In [ ]:
exp.show_in_notebook(text=newsgroups_test.data[idx], labels=(15,))

Глядя на этот пример, можно увидеть, что в заголовке или кавычках может быть и полезный сигнал, который будет помогать обобщению (например,  в строке «Тема»).

А есть и слова, которые нельзя обобщать (например, адреса электронной почты и названия учреждений).

##### $\color{brown}{\text{Допольнительная информация}}$

###### TF-IDF:


https://ru.wikipedia.org/wiki/TF-IDF

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a

 ###### Multinomial Naive Bayes

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html?highlight=multinomial%20naive%20bayes#sklearn.naive_bayes.MultinomialNB

## Другие библиотеки

[DEN](https://github.com/isaacrob/DEN) : основан на новом режиме обучения сиамской нейронной сети без учителя и функции потерь, который называется Differentiating EmbeddingNetworks (DEN).

Сиамская нейронная сеть находит отличительные или похожие черты между конкретными парами образцов в наборе данных и использует эти функции, чтобы встроить набор данных в пространство более низкой размерности, где он может быть визуализирован.

В отличие от существующих алгоритмов визуализации, таких как UMAP ort-SNE, DEN является параметрическим, то есть его можно интерпретировать такими методами, как SHAP.

[XAI](https://towardsdatascience.com/xai-build-your-own-deep-learning-interpretation-algorithm-6e471b59af7) 

[Boruta](https://github.com/scikit-learn-contrib/boruta_py) — это метод выбора всех релевантных функций, изобретенный Витольдом Р. Рудницки и разработанный Мироном Б. Курса из ICM UW.

Boruta выбирает все необходимые признаки, в то время как другие являются не оптимальными. Метод пытается найти все признаки, несущие информацию, используемую для прогнозирования, вместо того, чтобы находить компактное подмножество признаков, по которым некоторый классификатор имеет минимальную ошибку.

Зачем беспокоиться о выборе всех необходимых признаков?

Когда вы пытаетесь понять феномен, создавший ваши данные, вы должны заботиться о тех факторах, которые способствуют этому, а не только о самых грубых признаках этого в контексте вашей методологии. Но помните, что минимальный оптимальный набор признаков по определению зависит от вашего выбора классификатора. В примере ниже можно увеличить число слоев и, отвергнутые ранее признаки окажутся востребованными моделью.

[Boruta Feature Selection](https://towardsdatascience.com/simple-example-using-boruta-feature-selection-in-python-8b96925d5d7a) 

## Boruta 

Автоматический отбор признаков  [boruta_py](https://github.com/scikit-learn-contrib/boruta_py)

Попробуем "раздуть" наш датасет, добавив в него "теневые признаки" $-$ перемешанные реальные. Таким образом наш датасет точно будет содержать хорошие признаки (мы ничего не удаляем). 

### Идея

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/pdd/boruta1.png" alt="alttext" width=600/>

После этого обучаем модель и отбираем те признаки, которые по качеству лучше теневых. Успех? Не совсем, ведь перемешивание это случаный процесс. Надо повторить процедуру несколько раз для того чтобы удалить случайные скачки качества.

Таким образом для каждого признака мы будем знать сколько раз мы его отобрали. Получаем распределение. Самая большая неопределенность будет в середине (вероятность отобрать = 0.5):

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/pdd/boruta_bell.png" alt="alttext" width=700/>


https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a

Набор (в нашем случае из 20) испытаний Бернулли это биномиальное распределение. поступаем просто. Со значимостью допустим 0.05 берем все из хорошего хвоста и отбрасываем из плохого хвоста. С признаками из середины колокола ничего особо не сделаешь, увеличение числа итераций приведет к ужатию колокола но глобально не поможет.

Если нам нужна хорошо интерпретируемая модель то надо брать только "точно хорошие" признаки. А если мы готовы поднабрать мусорных признаков то можем отбросить только плохой хвост.

Классическая Борута работает с Gini Impurity, но есть и реализации с SHAP. В этом плане она универсальна. В принципе ее можно применять и для градиентного бустинга тоже.

### Применение

In [ ]:
from IPython.display import clear_output
!pip install boruta
clear_output()

Питоновская реализация Boruta соответствует  API sklearn и может использоваться как в конвейере, так и самостоятельно.

[Boruta Feature Selection (an Example in Python)](https://towardsdatascience.com/simple-example-using-boruta-feature-selection-in-python-8b96925d5d7a)

Загрузка датасета

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd

# load dataset
datasets = load_boston()
X = pd.DataFrame(datasets['data'], columns = datasets['feature_names'])
y = pd.Series(datasets['target'], name = 'target_values')
X[:2]

In [ ]:
X.shape, type(X), y.shape, type(y)

Разобьем датасет на обучающую и тестовую выборки

In [ ]:
from sklearn.model_selection import train_test_split
def split_data(X,y):
  global X_train, X_test, y_train, y_test
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  X_train = X_train.values
  X_test = X_test.values
  y_train = y_train.values
  y_test = y_test.values

  print("X_train",X_train.shape)
  print("X_test", X_test.shape)
  print("y_train", y_train.shape)
  print("y_test", y_test.shape)
  print("type of y_test", type(y_test))

split_data(X,y)

Обучим модель

In [ ]:
from sklearn.ensemble import RandomForestRegressor
def evaluate():
  # define random forest regressor
  forest = RandomForestRegressor(n_jobs=-1,  max_depth=4,random_state = 42)
  forest.fit(X_train, y_train)
  r2 = forest.score(X_test, y_test)
  # R2 is coefficient of determination 
  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
  print(f'R2 score {r2:.4f}')
  return forest
forest = evaluate()

Проведем оценку признаков при помощи Boruta.
P.S. Потребуется около 30 сек.

In [ ]:
from boruta import BorutaPy

# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2, random_state=42)

# find all relevant features
feat_selector.fit(X_train, y_train)


Выведем признаки с оценкой их важности и очистим датасет от маловажных признаков

In [ ]:
# zip my names, ranks, and decisions in a single iterable
feature_ranks = list(zip(datasets.feature_names, 
                         feat_selector.ranking_, 
                         feat_selector.support_))

# iterate through print out the results and remove features with low rank
for feat in feature_ranks:
    print('Feature: {:<25} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
    if feat[2] == False: del X[feat[0]]

Датасет без "лишних" по мнению Boruta признаков

In [ ]:
X[:2]

Обучение на признаках, отобранных Boruta
Давайте проверим, можем ли мы добиться такого же результата, если проанализируем и уберем «лишние» данные?

In [ ]:
 split_data(X,y)
 _ = evaluate()

Удалив 4 признака мы не потеряли в точности.

# Примеры explanations для разных видов данных

## Tabular examples

Для примера скачаем датасет жилья Бостона (boston_dataset), в котором проанализируем зависимость цены на жилье от параметров жилья и района, в котором оно находится.

In [ ]:
from IPython.display import clear_output
!pip install shap
clear_output()

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import shap

# load dataset
boston_dataset = load_boston()
X = pd.DataFrame(data=boston_dataset['data'], columns=boston_dataset['feature_names'])
y = boston_dataset['target']

# Split the data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Build the model with the random forest regression algorithm
model = RandomForestRegressor(n_estimators=10, max_depth=6, random_state=rng)
model.fit(X_train, y_train)

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

In [ ]:
# Можно посмотреть детальное описание датасета
print(boston_dataset.DESCR)
print(type(shap_values)) # numpy.ndarray

**Force plots** 

Хороший способ визуализировать вклад каждого фактора в конкретный прогноз — использовать график сил.

В приведенном ниже примере показан график силы для 2-го случая в тестовом наборе данных.

In [ ]:
# load JS visualization code to notebook
shap.initjs()
# visualize the first prediction’s explanation
shap.force_plot(explainer.expected_value, shap_values[1, :], X.iloc[1, :])

* f (x) или прогноз модели равен 20.56 по анализируемому обьекту недвижимости. А base_value - это средний прогноз по всему набору тестовых данных. Или, другими словами, это значение, которое можно было бы спрогнозировать, если бы мы не знали каких-либо характеристик текущего примера.

* Элементы, которые способствуют увеличению цены, показаны красным, а те, которые уменьшают — синим.

ПРИМЕР, может оказаться, что:
* LSTAT (% lower status of the population) оказывает сильное положительное влияние на цену дома и подталкивает прогноз вправо.
* Другими важными характеристиками, повышающими стоимость дома, являются НАЛОГИ и NOX (nitric oxides concentration (parts per 10 million)).
* А вот RM (average number of rooms per dwelling) оказывает сильное негативное влияние на цену жилья, за ней следует DIS (weighted distances to five Boston employment centres).

**Waterfall_plot**

Другой способ понимания влияния факторов для конкретного примера:

In [ ]:
# visualize the first prediction's explanation using waterfall
i = 1 # смотрим влияние факторов для 2го примера
features = list(X.columns) # формируем список признаков
class ShapObject:
    def __init__(self, base_values, data, values, feature_names):
        self.base_values = base_values # Single value
        self.data = data # Raw feature values for 1 row of data
        self.values = values # SHAP values for the same row of data
        self.feature_names = feature_names # Column names
        
shap_object = ShapObject(base_values = explainer.expected_value[0],
                         values = shap_values[i,:],
                         feature_names = features,
                         data = X[features].iloc[i,:])

shap.waterfall_plot(shap_object)

Этот график обьясняет движущие силы конкретного прогноза:

Влияние каждой отдельной функции (менее значимые функции объединяются в одну группу) представлено стрелками, которые перемещают логарифмическое отношение шансов влево и вправо, начиная с базового значения.

---

Приведенный выше пример приведен только для одного случая.

Если мы возьмем много пояснений Force plots, повернем их на 90 градусов, а затем сложим их по горизонтали, мы сможем увидеть объяснения для всего набора данных (в notebook этот график является интерактивным):

In [ ]:
# load JS visualization code to notebook
shap.initjs()
# visualize the training set predictions
shap.force_plot(explainer.expected_value, shap_values, X)

In [ ]:
import seaborn as sns
import pandas as pd

df = pd.DataFrame(shap_values, columns=X.columns)
sns.clustermap(df)
plt.show()

**Summary plot**

Сводный график с plot_type = 'bar' даст нам график важности переменной.

Функции с высокой предсказательной способностью показаны вверху, а с низкой предсказательной силой — внизу.

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type='bar')

Здесь мы только что рассмотрели алгоритм TreeExplainer для интерпретации модели.

Вы можете изучить остальные алгоритмы — DeepExplainer, kernelExplainer, LinearExplainer и GradientExplainer.

## NLP: Пример абстрактного обобщения текста 

потребуется время

https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/text.html

В этом примере интерпретируется модель генерации объяснений для предварительно обученной модели для составления краткого резюме статьи.

Используется датасет Extreme Summarization [XSum](https://huggingface.co/sshleifer/distilbart-xsum-12-6)

In [ ]:
!pip install datasets
!pip install transformers
clear_output()

In [ ]:
import shap
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-6")
model =  AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-xsum-12-6").to(device) 

 
dataset = load_dataset('xsum', split='train') # загружаем датасет
s = dataset['document'][0:1]                 # slice inputs from dataset to run model inference on
explainer = shap.Explainer(model, tokenizer)  # create an explainer object
explanation = explainer(s)                   # Compute shap values
clear_output()

In [ ]:
shap.initjs()

In [ ]:
shap.plots.text(explanation)                 # Visualize shap explanations

## Изображения

### LIME
[Local Interpretable Model-Agnostic Explanations (LIME): An Introduction](https://www.oreilly.com/content/introduction-to-local-interpretable-model-agnostic-explanations-lime/)

[Using Lime with Pytorch](https://github.com/marcotcr/lime/blob/master/doc/notebooks/Tutorial%20-%20images%20-%20Pytorch.ipynb)

#### Идея
<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/7_Google_Inception_neural_network.png" alt="alttext" width=750/>

Давайте разберемся, как работает такое эффектное представление областей, по которым нейронная сеть осуществляет предсказание класса.

На рисунке ниже показан пример того, как LIME работает для классификации изображений.

Представьте, что мы хотим объяснить классификатор, который предсказывает, насколько вероятно, что изображение содержит древесную лягушку.

Мы берем изображение слева и делим его на интерпретируемые компоненты (смежные [суперпиксели](https://darshita1405.medium.com/superpixels-and-slic-6b2d8a6e4f08)).

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/l10_figure3.jpg" alt="alttext" width=350/>

Далее мы отключаем некоторые из суперпикселей (закрашиваем серым).

Для каждой такой картинки мы получаем вероятность того, что на изображении есть древесная лягушка и формируем из датасет из частично закрашенных картинок и предсказаний.

Затем мы обучаем линейную модель на этом наборе данных, веса соответствующие суперпикселю будут объяснять его вклад в передсказание.

<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/l10_figure4.jpg" alt="alttext" width=650/>

Проанализируем предсказание сверточной сети Google Inception. Посмотрим, почему ее классификатор предсказывает «древесную лягушку» как наиболее вероятный класс, за которым следуют «бильярдный стол» и «воздушный шар» с более низкими вероятностями.

Мы видим, что классификатор в первую очередь фокусируется на морде лягушки как на объяснении предсказанного класса.

Это также проливает свет на то, почему «бильярдный стол» имеет ненулевую вероятность: руки и глаза лягушки напоминают бильярдные шары, особенно на зеленом фоне. Точно так же сердце похоже на красный воздушный шар.


<img src="http://edunet.kea.su/repo/src/L10_Explainability/img/l10_figure6.jpg" alt="alttext" width=650/>

#### Анализ ResNet18

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
#import numpy as np
import os, json

#import torch
from torchvision import models, transforms
import torch.nn.functional as F

In [ ]:
!wget --no-check-certificate 'http://edunet.kea.su/repo/src/L10_Explainability/data/cat_and_dog1.jpg' -O cat_and_dog1.jpg
!wget --no-check-certificate 'http://edunet.kea.su/repo/src/L10_Explainability/data/cat_and_dog2.png' -O cat_and_dog2.png
!wget --no-check-certificate 'http://edunet.kea.su/repo/src/L10_Explainability/data/imagenet_class_index.json' -O imagenet_class_index.json

In [ ]:
def get_image(path):
    with open(os.path.abspath(path), 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB') 
        
img = get_image('cat_and_dog1.jpg')
plt.imshow(img)

Теперь нам нужно преобразовать это изображение в тензор Pytorch, а также его нормализовать для использорвания в нашей предварительно обученной модели.

In [ ]:
# resize & normalize
def get_input_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])       
    transf = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop((224, 224)),
        transforms.ToTensor(),
        normalize])    
    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    # unsqeeze converts single image to batch of 1
    return transf(img).unsqueeze(0)


Загрузим предобученную модель Resnet18, доступную в Pytorch и классы изображений из ImageNet.

In [ ]:
model = models.resnet18(pretrained=True)

idx2label, cls2label, cls2idx = [], {}, {}
with open(os.path.abspath('/content/imagenet_class_index.json'), 'r') as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
    cls2label = {class_idx[str(k)][0]: class_idx[str(k)][1] for k in range(len(class_idx))}
    cls2idx = {class_idx[str(k)][0]: k for k in range(len(class_idx))}

Получим предсказание. А после этого полученные нами прогнозы (логиты), пропустим это через softmax, чтобы получить вероятности и метки классов для 5 лучших прогнозов.

In [ ]:
img_t = get_input_tensors(img)
model.eval()
logits = model(img_t)

probs = F.softmax(logits, dim=1)
probs5 = probs.topk(5)
plt.imshow(img)
tuple((p,c, idx2label[c]) for p, c in zip(probs5[0][0].detach().numpy(), probs5[1][0].detach().numpy()))

(tabby - это тоже кошка.)

Применим LIME

In [ ]:
!pip install lime

Lime генерирует массив изображений из исходного входного изображения с помощью алгоритма пертубации.

Таким образом, нам нужно предоставить конструктору :
1. Исходное изображение в виде массива numpy
2. Функцию классификации, которая будет принимать массив искаженных изображений в качестве входных данных и генерировать вероятности для каждого класса для каждого изображения в качестве выходных.



Поэтому потребуется вспомогательная функция для обработки пакета изображений, в соответствии с API LIME.

In [ ]:
import torch
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batch_predict(images): # images are numpy arrays
    model.eval()
    transf = get_input_transform()
    batch = torch.stack(tuple(transf(Image.fromarray(i)) for i in images), dim=0)

    model.to(device)
    batch = batch.to(device)
    
    logits = model(batch)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

Создадим экзепляр ImageExplainer и сгенерируем объект explanation

In [ ]:
import lime
import numpy as np
from lime import lime_image

explainer = lime_image.LimeImageExplainer(random_state= 42)
explanation = explainer.explain_instance(np.array(img.resize((224,224))), # Lime assume that input is a numpy array :(
                                         batch_predict, # classification function
                                         top_labels=5, 
                                         hide_color=0, 
                                         num_samples=1000,  # number of images that will be sent to classification function
                                         random_seed = 42)

Выведем top5 предсказаний сделанных через LIME.

P.S. Они не обязанны совпадать с предсказаниями для картинки без изменений.

In [ ]:
for i, id in enumerate(explanation.top_labels):
  print(i,idx2label[id])

Воспользуемся маской на изображении и посмотрим области, которые дают лучший прогноз.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))
from skimage.segmentation import mark_boundaries
for i, id in enumerate(explanation.top_labels[:2]):
  temp, mask = explanation.get_image_and_mask(id, positive_only=False, num_features=5, hide_rest=False)
  img_boundry = mark_boundaries(temp, mask)
  ax[i].imshow(img_boundry)
  ax[i].set_title(idx2label[id])
  # количество кластеров, которые нужно показать на рисунке: num_features=5
  # показать или нет отрицательно влияющие кластеры: positive_only=False 
  # cреди первых 5-ти отрицательных может не оказаться

Зеленым цветом обозначена область наивысшего прогноза и оранжевым — области, которые меньше всего соответствуют нашему прогнозу. При `positive_only=False` будут показанны только границы.

И другое изображение:

In [ ]:
img2 = get_image('cat_and_dog2.png')
plt.imshow(img2)

Запуск Lime

In [ ]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(np.array(img2.resize((224,224))), 
                                         batch_predict, # classification function
                                         top_labels=5, 
                                         hide_color=0, 
                                         num_samples=1000) # number of images that will be sent to classification function
# Display top labels
for i, id in enumerate(explanation.top_labels):
  print(i,idx2label[id])

Выведем сегменты наиболее повлиявшие на каждое предсказание

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(50, 10))
from skimage.segmentation import mark_boundaries
for i, id in enumerate(explanation.top_labels):
  temp, mask = explanation.get_image_and_mask(id, positive_only=False, num_features=5, hide_rest=False)
  img_boundry = mark_boundaries(temp, mask)
  ax[i].imshow(img_boundry)
  ax[i].set_title(idx2label[id])

В то время как большинство методов, описанных в первом сообщении блога, являются глобальными методами, LIME — это метод, имеющий локальную область применения.

Таким образом, LIME позволяет нам объяснить конкретные прогнозы любого классификатора.

Структура LIME является гибкой в ​​том смысле, что любую интерпретируемую модель можно использовать для объяснения прогнозов.

Кроме того, концепция интерпретируемых компонентов позволяет применять LIME для обьяснения данных большой размерности, таких как классификация изображений или текста.

##### $\color{brown}{\text{Допольнительная информация}}$

###### Shap.PartitionExplainer  

В Shap есть класс аналогичный LimeImageExplainer: 


[shap.PartitionExplainer](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.PartitionExplainer.html)

Однако его запуск с Pytorch моделями приводит к ошибке:

https://github.com/slundberg/shap/issues/1969


которая не была устраненна на момент составления блокнота. Код которым он должен запу

In [ ]:
!pip install shap

In [ ]:
import shap

# define a masker that is used to mask out partitions of the input image. 
masker = shap.maskers.Image("inpaint_telea", (224,224,3))

# create an explainer with model and image masker 
explainer = shap.Explainer(batch_predict, masker, output_names=list(cls2label.values())) 


# Here we explain one images using 5 evaluations of the underlying model to estimate the SHAP values
explanation = explainer([np.array(img.resize((224,224)))], max_evals=5, batch_size=2, outputs=shap.Explanation.argsort.flip[:1])#, outputs=shap.Explanation.argsort.flip[:2])

Код вниже ызывает exception "Labels must have same row count as shap_values arrays!"

[https://github.com/slundberg/shap/issues/1969](https://github.com/slundberg/shap/issues/1969)

In [ ]:
shap.image_plot(explanation)

### Gradient Ascent

Имея информацию о структуре модели, можно использовать градиент что бы понять как входы связанны с выходом.


Вместо того что бы случайным образом закрашивать части изображения и много раз запускать модель, мы можем один раз посчитать градиент по входному изображению, и понять какие пикселы оказали наибольшее влияние на предсказание модели.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-17.png" width="700">



Воспользуемся изображением из предидущего примера

In [ ]:
img_t.requires_grad = True # Tell pytorch to compute grads w.r.t. inputs too
logits = model(img_t.to(device)) # [1,1000] batch of one element, 1000 class scores
top_score, top_idx = logits[0].topk(1) # Get id of class with best score
id = top_idx[0].item()
print(id, idx2label[id]) # Print the label this class 
score = logits[0][id] # Model output for paticular class



Для выхода модели соответствующему нашему классу рассчитываем градиент.

In [ ]:
# Compute gradients

score.backward(retain_graph=True) 

# retain_grad = True is not nessesary
# But if we run this code second time, we got a torch error without it
# because pytorch want to accumulate gradients explicitly 

print(img_t.grad.shape)
print(img_t.grad[0,:3,:3]) # Show some parts d_Image/d_score

У входного тензора (изображения) появился градиент который указывает на то как каждый элемент повлиял на выход модели. Отобразим этот градиент.

In [ ]:
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

# Helper method to display grad
def grad_to_image(raw_grads):
    grad_of_first_batch_element = raw_grads[0]
    # Summ grads of RGB channes
    smap = torch.sum(grad_of_first_batch_element, dim=0) 
    # Translate raw grad values to byte [0 .. 255] for displaying
    max_val = smap.max()
    img = (smap / max_val) * 255
    # Filter using threshold to make image sharp
    img[img < 0] = 0 
    img[img > 50] = 255
    return img.numpy().astype(int)

sailency_map = grad_to_image(img_t.grad)

plt.subplot(1, 2, 1)
imshow(img )
plt.subplot(1, 2, 2)
imshow(sailency_map)



##### $\color{brown}{\text{Допольнительная информация}}$

###### Adversarial attacks

Принцип взятия градиента по входу используется при состязательных атаках (adversarial attacks).

Если не просто визуализировать градиент, а с его помощью менять изображение, то можно обмануть сеть и заставить ее неверно классифицировать картинку незначительно поменяв ее.


<img src ="http://edunet.kea.su/repo/src/L10_Explainability/img/gan/adv_attack.jpg" width="700">


Подробнее:
https://en.wikipedia.org/wiki/Adversarial_machine_learning
https://blog.mlreview.com/the-intuition-behind-adversarial-attacks-on-neural-networks-71fdd427a33b


Пиксели получившие большой градиент находится именно в том месте где изображенна собака - это хорошо. 

При этом есть группы пикселей вверху изображения которые тоже подсвеченны, но относятся к фону  - это плохо. Значит модель может ошибаться из-за такого рода фона.


На этом принципе посторен модуль [shap.DeepExplainer](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html).

Он уже не зависим от модели(not Model agnostic) так как использует градиенты, зато позволяет оценить связи более точно.


### SHAP Deep Explainer



DeepExplainer релизует алгоритм [DeepLift](https://arxiv.org/abs/1704.02685) который базируется на идее GradientAscending но не ограничивается ей. Алгоритм использует надобор произвольных изображений из датасета.  

Для сетей обученных на ImageNet авторы предлагают имспользовать `shap.datasets.magenet50` :

`The point is to have a random sample of ImageNet for use as a background distribution for explaining models trained on ImageNet data.`

что мы и сделаем.



In [ ]:
img_t = get_input_tensors(img)
model.eval()
logits = model(img_t)

probs = F.softmax(logits, dim=1)
probs5 = probs.topk(5)
plt.imshow(img)
tuple((p,c, idx2label[c]) for p, c in zip(probs5[0][0].detach().numpy(), probs5[1][0].detach().numpy()))

In [ ]:
!pip install shap

В SHAP есть встроенный фрагмент датасета ImageNet (50 изображений) воспользуемся им.

Метки не корректны:
https://shap-lrjball.readthedocs.io/en/latest/api.html?highlight=datasets.imagenet50#shap.datasets.imagenet50

поэтому проигнорируем их

In [ ]:
import shap
import matplotlib.pyplot as plt
import json

imagenet_50, broken_targets = shap.datasets.imagenet50()

print("Data shape", imagenet_50.shape,type(X))
# Show first image
plt.imshow(imagenet_50[0].astype('int')) 

Можно взглянуть и на остальные картинки

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=10, figsize=(25, 10))
for i,img in enumerate(X):
  row = i // 5
  col = i % 5
  ax[col,row].imshow(img.astype('int'))



In [ ]:
# for performance reason use as background only 10 images in PyTroch format
background = torch.tensor(imagenet_50[0:10]).permute(0,3,1,2).to(device)/255 

# https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html
explainer = shap.DeepExplainer(model.to(device), background)
shap_values = explainer.shap_values(img_t) # List

По умолчанию возвращаются shap индексы для каждого класса, для каждого пикселя 

In [ ]:
print("Classes", len(shap_values)) 
print("Valuse",shap_values[0].shape)

Для отображения результатов используем метод [shap.image_plot](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.image_plot.html?highlight=image_plot)

его API ждет данные в виде списков numpy - массивов, поэтому нам потребуется преобразовать данные.


In [ ]:
import numpy as np

# Get indexes of top5 classes predicted by model
top5_indexes = probs5.indices.squeeze(0).numpy().astype('int')

# Get shap vlues for this classes
shap_values_for_top_results = np.array(shap_values)[top5_indexes]

# Move color channels back for numpy compability (...,3,224,224 ) - > (...,224,224,3)
shap_values_for_top_results = np.swapaxes(shap_values_for_top_results, 4, 2)  # swapaxes is do the same thing as torch.permute

# Convert first dim of numpy.aray back to pythol list as required sahp.image_plot api
shap_values_for_top_results = list(shap_values_for_top_results[:])

# Prepare test image
test_image = np.array(img.resize((224,224))) # resize to size of shap values
test_image = test_image[np.newaxis, ...]/255


# Get lagels for top5 classes

shap_labels = np.array(idx2label)[top5_indexes]
shap_labels = [list(shap_labels)] # One list for sample

print("Len of shap_values list",len(shap_values_for_top_results)) # number of classes to explain
print("Shape of one value",shap_values_for_top_results[0].shape) # n_samples, H,W,C
print(shap_labels) # n_samples, number of classes

Теперь визуализируем результаты:

In [ ]:
shap.image_plot(shap_values_for_top_results, test_image,labels = shap_labels)

На практике чаще всего интересны объяснения только для нескольких классов с максимальной уверенностью.

И в документации описан параметр который позволяет возвращать объяснения только для этих классов.

https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html

`ranked_outputs = 5, output_rank_order ='max'`

При этом возвращается кортеж:
shap_values и индексы классов для которых получено объяснение.



In [ ]:
shape_values_for_best_pred , indexes = explainer.shap_values(img_t,ranked_outputs = 5, output_rank_order ='max') # List

Результаты снова надо преобразовать их Pytorch формата

In [ ]:
# Convert shap val frop PyTorch format
shape_values_for_best_pred = np.swapaxes(np.array(shape_values_for_best_pred), 4, 2)
shape_values_for_best_pred = list(shape_values_for_best_pred[:])

# Get names for returned indexes
shap_labels = np.array(idx2label)[indexes.cpu()[0]]
shap_labels = [list(shap_labels)] # One list for sample


Визуализируем результат

In [ ]:

shap.image_plot(shape_values_for_best_pred , test_image, labels = shap_labels)

# 3. Список литературы

### Статьи

[How to Interpret Machine Learning Models with SHAP](https://www.youtube.com/watch?v=ylytZegK--I)

[Бесплатный курс от Kaggle: Machine Learning Explainability](https://www.kaggle.com/learn/machine-learning-explainability)

[EXPLAINABLE AI IN CREDIT RISK MANAGEMENT](https://arxiv.org/pdf/2103.00949v1.pdf)

[Predicting Driver Fatigue in Automated Driving with Explainability](https://arxiv.org/pdf/2103.02162v1.pdf)

[Fooling LIME and SHAP: Adversarial Attacks on Post hoc Explanation Methods](https://arxiv.org/pdf/1911.02508v2.pdf)

[Interpretable Machine Learning]
(https://christophm.github.io/interpretable-ml-book/intro.html)

### SHAP
[Welcome to the SHAP documentation](https://shap.readthedocs.io/en/stable/index.html)

[Git](https://github.com/slundberg/shap)

[A Unified Approach to Interpreting Model Predictions](https://arxiv.org/pdf/1705.07874v2.pdf)

[SHAP (SHapley Additive exPlanations)](https://christophm.github.io/interpretable-ml-book/shap.html)

### LIME

[“Why Should I Trust You?” Explaining the Predictions of Any Classifier](https://arxiv.org/pdf/1602.04938.pdf)

[What does LIME really see in images?](https://arxiv.org/pdf/2102.06307v1.pdf)

[Git](https://github.com/marcotcr/lime)

### BORUTA
[Feature Selection with the Boruta Package](https://www.jstatsoft.org/index.php/jss/article/view/v036i11/v36i11.pdf)

[Boruta Explained Exactly How You Wished Someone Explained to You](https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a)

# 4. Заключение:
---
* В ходе урока мы убедились в важнности интерпретации работы моделей Искусственного Интеллекта.
* Рассмотрели основные библиотеки, которые используются для интерпретирования: SHAP и LIME. А также библиотеку BORUTA, которая позволяет проводить анализ влияния признаков датасета на качество обучения. И дает рекомендации по исключению тех, которые влияют на результата отрицательно.
* Рассмотрели применение бибилиотек на примерах:
    - Табличные данные
    - NLP (машинный перевод текста, создание резюме статьи и классификации текстов)
    - CV

Пренебрежение обьяснением того, почему модель дала тот или иной результат, ведет к недоверию не только к самой модели, но и к конкретным прогнозам. А, следовательно, является существенным препятствием для дальнейшего введения Вашей идеи в production.


**Помните об этом!**
